In [ ]:
# --- IMPORTS ---
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

# --- LOAD MODEL FROM JSON + WEIGHTS ---
with open("model_architecture.json", "r") as json_file:
    model_json = json_file.read()

model = model_from_json(model_json)
model.load_weights("model_weights.h5")

print(" Model loaded successfully for prediction.")

# --- PREPARE TEST DATA ---
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    "dataset_path/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# --- PREDICTION ---
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_labels = test_generator.classes

# --- ACCURACY & CONFUSION MATRIX ---
accuracy = accuracy_score(true_labels, predicted_classes)
cm = confusion_matrix(true_labels, predicted_classes)

print(f"\n Test Accuracy: {accuracy * 100:.2f}%")
print("\n Confusion Matrix:")
print(cm)
